#### In this file:

1) Webscrap the magicformulainvesting website, to extract the table with the top 50 firms (with market cap above 50M) 


2) Import the prestige dataset, update the information, and check whether there are or not new stocks to buy and old ones to sell    


3) Send an email with a report: stocks to buy and stocks to sell


4) Update the mf_historical dataset


In [1]:
!pip install prettyplotlib pretty_html_table

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 697 kB 18.5 MB/s eta 0:00:01
  Created wheel for prettyplotlib: filename=prettyplotlib-0.1.7-py3-none-any.whl size=16753 sha256=7d94d7b1cf8bd8b239d9c97134f6d7f3b420e767a87032582dfdbe4756ee7c90
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0d/a5/f0/c1f6e8094c7b40e9731adfb4a76538be070274ec8d7c2c7758
Successfully built prettyplotlib


In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

#to import datasets on the IBM Cloud:
import types
from botocore.client import Config
import ibm_boto3

#to save datasets on the IBM Cloud:
from project_lib import Project
project = Project(project_id='Not Actual Code', project_access_token='Not Actual Codee')
pc = project.project_context

#to send email:
import smtplib,ssl
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders
from pretty_html_table import build_table

import io

import ast

## 1. Webscrap the table from the magicformulainvesting.com website

In [3]:
#Your personal information:

#your magicformulainvesting email and password:
email_mf = 'jlbgouveia@gmail.com'
password_mf = 'Not Actual Code'

#your email gmail account information:
send_from = 'dumontcapital94@gmail.com'
password = 'Not Actual Code'
send_to = 'dumontcapital@gmail.com'

In [4]:
login_data = {'Email': email_mf,
'Password': password_mf,
'login': 'Login'}

conditions = {'MinimumMarketCap': '50',
'Select30': 'false',
'stocks': 'Get Stocks'}

In [5]:
#Because we are loggin in, we need to create a session:
with requests.Session() as s:
    url = 'https://www.magicformulainvesting.com/Account/Logon'
    r = s.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    
    r = s.post(url, data = login_data) #logging in
    
    #Now we need to request the top 50, with market cap above 50
    r = s.post( 'https://www.magicformulainvesting.com/Screening/StockScreening',data = conditions)

##### Now that we are logged in on the desired page, we can start the web scrapping

In [6]:
#Information we will scrap:
company_name = []
company_ticker = []
market_cap = []
date_price = []
most_recent_quarter_data = []

soup = BeautifulSoup(r.text, 'html.parser')
#appending odd_rows:
for i in soup.find_all('tr', attrs = {'class':"altrow"}):
    company_name.append(i.find_all('td')[0].text.strip())
    company_ticker.append(i.find_all('td')[1].text.strip())
    market_cap.append(i.find_all('td')[2].text.strip())
    date_price.append(i.find_all('td')[3].text.strip())
    most_recent_quarter_data.append(i.find_all('td')[4].text.strip())
    
#For the even rows, the website maintained the class as class = "", which will make us select more information than needed when 
#performing the soup.find_all('tr', attrs = {'class':""}) - so, we'll need to use conditional statements:

for i in soup.find_all('tr', attrs = {'class': ""}):
    if len(i.find_all('td')) == 5:
        company_name.append(i.find_all('td')[0].text.strip())
        company_ticker.append(i.find_all('td')[1].text.strip())
        market_cap.append(i.find_all('td')[2].text.strip())
        date_price.append(i.find_all('td')[3].text.strip())
        most_recent_quarter_data.append(i.find_all('td')[4].text.strip())

#### Now Let's turn this information into a pandas dataframe

In [7]:
df_mf = pd.DataFrame({'company name': company_name, 
              'ticker': company_ticker, 
              'price date': date_price,
              'most recent quarter data': most_recent_quarter_data})
df_mf = df_mf.sort_values('company name').reset_index(drop = True) #sorting by company name

In [8]:
today = datetime.combine(date.today(), datetime.min.time())
df_mf['report date'] = today #today's date, in datetime, not including time

In [9]:
df_mf

,company name,ticker,price date,most recent quarter data,report date
0,AMC Networks Inc,AMCX,06/07,03/31,2021-06-08
1,Academy Sports and Outdoors Inc,ASO,06/07,01/31,2021-06-08
2,Altria Group Inc,MO,06/07,03/31,2021-06-08
3,Best Buy Co Inc,BBY,06/07,04/30,2021-06-08
4,Blueprint Medicines Corp,BPMC,06/07,03/31,2021-06-08
5,Buckle Inc. (The),BKE,06/07,04/30,2021-06-08
6,Catalyst Pharmaceutical Inc,CPRX,06/07,03/31,2021-06-08
7,Cigna Corp,CI,06/07,03/31,2021-06-08
8,Citi Trends Inc,CTRN,06/07,04/30,2021-06-08
9,Co-Diagnostic Inc,CODX,06/07,12/31,2021-06-08


## 2. Import 'prestige' dataset, update it and check for stocks to buy/sell

In [10]:
#Function to import files from IBM Cloud

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b8784d29aa5493a8b1aabf94dfcda80 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='f3w6UsdAqcUncvkiUCksxwb7S5Jmd1pUMcGKt3wUDOgG',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

In [11]:
#import prestige dataset:

body = client_3b8784d29aa5493a8b1aabf94dfcda80.get_object(Bucket='magicfomula-donotdelete-pr-fsblofvklbtag3',Key='prestige.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

prestige = pd.read_csv(body)
prestige.set_index('report date', inplace = True)
prestige.index = pd.to_datetime(prestige.index)
prestige.tail()

,top 50,top week trailing,bronze,sell bronze
report date,,,,
2021-06-03,"['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...","['AMCX', 'ASO', 'APT', 'MO', 'BDSI', 'BKE', 'C...",['JAZZ'],['DKS']
2021-06-04,"['AMCX', 'ASO', 'MO', 'BBY', 'BPMC', 'BKE', 'C...","['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BDSI', 'B...","['COLL', 'ZUMZ']",['BBY']
2021-06-05,"['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...","['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...",NaN,NaN
2021-06-06,"['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...","['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...",NaN,NaN
2021-06-07,"['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...","['AMCX', 'ASO', 'APT', 'MO', 'BBY', 'BPMC', 'B...",NaN,NaN


In [12]:
#inserting top 50 of the day
prestige.loc[today, 'top 50'] = str([*df_mf['ticker']])

In [13]:
#getting the firms that were in the top 50 during the week (in list format)
top_week = [*prestige.loc[today + relativedelta(days = -7): today + relativedelta(days = -1),'top 50'].apply(ast.literal_eval).apply(pd.Series).stack()]
prestige.loc[today, 'top week trailing'] = str(top_week)

#checking for 'bronze' stocks (i.e., if there are any new stocks relative to last week, and insert in 'bronze' column)
bronze = [*set(ast.literal_eval(prestige.loc[today,'top 50'])).difference(ast.literal_eval(prestige.loc[today,'top week trailing']))]
if len(bronze) > 0:
    prestige.loc[today,'bronze'] = str(bronze)
else: prestige.loc[today,'bronze'] = np.nan
    
# selling stocks from last week
prestige.loc[today, 'sell bronze'] = prestige.loc[today + relativedelta(days = -7), 'bronze']

In [14]:
# saving the prestige dataframe
project.save_data(file_name = "prestige.csv",data = prestige.to_csv(index=True), overwrite = True)

{'file_name': 'prestige.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'magicfomula-donotdelete-pr-fsblofvklbtag3',
 'asset_id': '98e9500f-2c41-46ff-a82f-3c8aa0772055'}

### 3. Send email with stocks to buy and sell

First, we need to build the dataframe with the stocks to buy and to sell

In [15]:
#Build a simple multiindex dataframe
cols = [np.array(['Buy', 'Buy','Sell','Sell']),
np.array(['Ticker', 'Company Name', 'Ticker', 'Company Name'])]

buy_sell_df = pd.DataFrame(columns = cols)

In [16]:
buy_sell_df

Empty DataFrame
Columns: [(Buy, Ticker), (Buy, Company Name), (Sell, Ticker), (Sell, Company Name)]
Index: []

In [17]:
long = {'Ticker': [], 'Company Name': []}
if type(prestige.loc[today, 'bronze']) == float:
    long['Ticker'] = '-'
    long['Company Name'] = 'No companies to buy today'
else:
    for ticker in ast.literal_eval(prestige.loc[today, 'bronze']):
        long['Ticker'].append(ticker)
        long['Company Name'].append(df_mf.loc[df_mf['ticker']==ticker, 'company name'].to_string())
        
short = {'Ticker': [], 'Company Name': []}
if type(prestige.loc[today, 'sell bronze']) == float:
    short['Ticker'] = '-'
    short['Company Name'] = 'No companies to sell today'
else:
    for ticker in ast.literal_eval(prestige.loc[today, 'sell bronze']):
        short['Ticker'].append(ticker)
        short['Company Name'].append(df_mf.loc[df_mf['ticker']==ticker, 'company name'].to_string())

In [18]:
if long['Ticker'] == '-':
    buy_sell_df.loc[0,'Buy'] = ['-', 'No companies to buy today']
else:
    for i in range(len(long['Ticker'])):
        buy_sell_df.loc[i,'Buy'] = [long['Ticker'][i], long['Company Name'][i]]

if short['Ticker'] == '-':
    buy_sell_df.loc[0,'Sell'] = ['-', 'No companies to sell today']
else:
    for i in range(len(short['Ticker'])):
        buy_sell_df.loc[i,'Sell'] = [short['Ticker'][i], short['Company Name'][i]]

In [19]:
buy_sell_df

Buy                                  Sell                                
  Ticker                   Company Name Ticker                    Company Name
0     LB             24    L Brands Inc   VYGR  47    Voyager Therapeutics Inc
1  TPRFF  18    Gran Colombia Gold Corp    NaN                             NaN

In [20]:
#email content:
subject = 'The Prestige Trading Strategy: relatório diário'
#body = """Good morning, \n
#Please find the stocks you should buy and sell according to the Prestige strategy. Please note that you should only sell these stocks if you bought them the week before. \n
#Kind regards, \n
#Zeza """

body = """\
<html>
  <head></head>
  <body>
    <p>Boa tarde,<br><br>
       Na tabela em baixo podes encontrar as ações para comprar/vender de acordo com a estratégia de investimento <i>Prestige<i>. Caso na semana passada não tenhas comprado as ações que diz para vender, por favor ignora (não fazer short selling).<br><br>
       Cumprimentos,<br>
       Zeza
    </p>
  </body>
</html>
"""


#sending through gmail:
server = 'smtp.gmail.com'
port = 587

In [21]:
#Create function to send email:

def send_dataframe(df, isTls = True):
    
    multipart = MIMEMultipart('alternative')
    multipart['From'] = send_from
    multipart['To'] = send_to
    multipart['Subject'] = subject

    multipart.attach(MIMEText(body + build_table(df, 'grey_light'), 'html'))
    
    smtp = smtplib.SMTP(server, port)
    if isTls:
        smtp.starttls()
    smtp.login(send_from,password)
    smtp.sendmail(send_from, send_to, multipart.as_string())
    smtp.quit()

In [22]:
#Send the desired dataframe
send_dataframe(buy_sell_df)

### 4. Update the historical dataset

In [23]:
#import mf_historical dataset:

body = client_3b8784d29aa5493a8b1aabf94dfcda80.get_object(Bucket='magicfomula-donotdelete-pr-fsblofvklbtag3',Key='mf_historical.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

mf_historical = pd.read_csv(body)
mf_historical.set_index('report date', inplace = True)
mf_historical.index = pd.to_datetime(mf_historical.index)

In [24]:
mf_historical.tail()

,company name,ticker,price date,most recent quarter data
report date,,,,
2021-06-07,ViacomCBS Inc,VIAC,06/04,03/31
2021-06-07,Vontier Corp,VNT,06/04,03/31
2021-06-07,Voyager Therapeutics Inc,VYGR,06/04,03/31
2021-06-07,Waitr Holdings Inc,WTRH,06/04,03/31
2021-06-07,Zumiez Inc,ZUMZ,06/04,04/30


In [25]:
#updating the historical information:

#adding a conditional statement, just in case I run the code more than once in a day:
if today in mf_historical.index:
    mf_historical.loc[today] = df_mf.set_index('report date')
else:
    mf_historical = pd.concat([mf_historical, df_mf.set_index('report date')])

In [26]:
#saving the mf_historical file
project.save_data(file_name = "mf_historical.csv",data = mf_historical.to_csv(index=True), overwrite = True)

{'file_name': 'mf_historical.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'magicfomula-donotdelete-pr-fsblofvklbtag3',
 'asset_id': '98d491ed-2b33-4d55-a426-0a674eda5411'}

Sources:   
Logging in into the website: [Indian Pythonista youtube video](https://www.youtube.com/watch?v=fmf_y8zpOgA)   
Web-Scrapping: [Computer Science web scraping youtube video](https://www.youtube.com/watch?v=Cb_5A6geOUw&list=LL&index=2)   
Sending DataFrame on Excel file through gmail: [snooze92's answer on stackoverflow](https://stackoverflow.com/questions/39999964/send-email-with-a-pandas-dataframe-as-attachment/49298990)